<a href="https://colab.research.google.com/github/garbamoussa/10-steps-to-become-a-data-scientist/blob/master/Problem_Set_3_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Problem Set 3 — The Proceedings of the Old Bailey**

In [ ]:
# mount 
from google.colab import drive
drive.mount('/content/drive')
# Dependencies
from datascience import *
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
import itertools
import seaborn as sn
import pandas as pd
import xml.etree.cElementTree as ET #XML Parser
from pathlib import Path 
from lxml import etree #ElementTree and lxml allow us to parse the XML file.
import requests #make request to server
import time #pause loop
from bs4 import BeautifulSoup
%matplotlib inline

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/datascience/tables.py:17: MatplotlibDeprecationWarning: The 'warn' parameter of use() is deprecated since Matplotlib 3.1 and will be removed in 3.3.  If any parameter follows 'warn', they should be pass as keyword, not positionally.
  matplotlib.use('agg', warn=False)
/usr/local/lib/python3.7/dist-packages/datascience/util.py:10: MatplotlibDeprecationWarning: The 'warn' parameter of use() is deprecated since Matplotlib 3.1 and will be removed in 3.3.  If any parameter follows 'warn', they should be pass as keyword, not positionally.
  matplotlib.use('agg', warn=False)


# **1. Classification**

For my date range in which to process trials, I intend to look at the period during the Revolutionary War between the United States and Great Britain, as I am interested in seeing how this historical event may have affected crime. Further, I initend to classify between verdict categories.


***(2) Load and parse the trials in your date range. This will entail iterating over the files in the
date range you’re interested in, parsing them with BeautifulSoup, and extracting each <div1
type="trialAccount id="...">...</div1> element.***


In [ ]:
import os
path = '/content/drive/MyDrive/Colab Notebooks/Problem Set 3/data/sessionsPapers'
sessions_dir = os.listdir(path)
sessions_dir[:10]

['18260112.xml',
 '18511215.xml',
 '18980620.xml',
 '17751018.xml',
 '18610819.xml',
 '18630511.xml',
 '18950617.xml',
 '18730505.xml',
 '18070408.xml',
 '19130107.xml']

In [ ]:
start_yr = 1750
end_yr = 1790
trial_session_xmls = []
trials_needed = [] 
for year in range(start_yr, end_yr + 1):
    for month in range(1, 13):
        for day in range(1, 32):
            file_name = f"/content/drive/MyDrive/Colab Notebooks/Problem Set 3/data/sessionsPapers/{year}{month:02}{day:02}.xml"
            try:
                infile = open(file_name).read()
                trial_session_xmls.append(file_name)
            except:
                continue

trial_session_xmls

xml_file = open(f'/content/drive/MyDrive/Colab Notebooks/Problem Set 3/data/sessionsPapers/17770115.xml')
BeautifulSoup(xml_file)

<?xml version="1.0" encoding="UTF-8"?><html><body><tei.2>
<text>
<div0 id="17770115" type="sessionsPaper">
<interp inst="17770115" type="collection" value="BAILEY"></interp>
<interp inst="17770115" type="year" value="1777"></interp>
<interp inst="17770115" type="uri" value="sessionsPapers/17770115"></interp>
<interp inst="17770115" type="date" value="17770115"></interp>
<xptr doc="17770115" type="transcription"></xptr>
<div1 id="f17770115-1" type="frontMatter">
<interp inst="f17770115-1" type="collection" value="BAILEY"></interp>
<interp inst="f17770115-1" type="year" value="1777"></interp>
<interp inst="f17770115-1" type="uri" value="sessionsPapers/17770115"></interp>
<interp inst="f17770115-1" type="date" value="17770115"></interp>
<xptr doc="177701150001" type="pageFacsimile"></xptr>THE WHOLE PROCEEDINGS ON THE KING's Commission of the Peace, Oyer and Terminer, and Gaol Delivery for the CITY of LONDON; AND ALSO The Gaol Delivery for the COUNTY of MIDDLESEX; HELD AT JUSTICE HALL in t

***(3) Extracting features from XML:***

In [ ]:
def case_date(case_soup):
    for element in case_soup.body.div1.contents:
        if element.name == 'interp':
            if element.attrs['type'] == 'date':
                return element.attrs['value']

def people_in_case(case_soup):
    people = []
    for persName in case_soup.body.div1.find_all('persname'):
        person = {}
        person["type"] = persName.attrs.get("type") # `thing.get(key)` is like `thing[key]` but returns None if key is not in x instead of raising an exception
        for interp in persName.find_all('interp'):
            fieldName = interp.attrs["type"]
            fieldValue = interp.attrs["value"]
            person[fieldName] = fieldValue
        people.append(person)
    return people
  
def case_descriptions(case_soup):
    descriptions = {}
    for rs in case_soup.find_all('rs'):
        desc = {}
        for interp in rs.find_all('interp'):
            fieldName = interp.attrs["type"]
            fieldValue = interp.attrs["value"]
            desc[fieldName] = fieldValue
        desc["text"] = rs.text.strip() # myString.strip() removes the whitespace at the beginning and end of myString
        descriptions[rs.attrs['type']] = desc
    return descriptions

In [ ]:
from math import nan
trial_ids = []
def table_of_cases(xml_file_names):
    rows = []
    for xml_file in xml_file_names:
        with open(f'{xml_file}', "r") as xml_file:
            case = BeautifulSoup(xml_file)
        people = people_in_case(case)
        date = case_date(case)
        descriptions = case_descriptions(case)
        row = {
            "date": date,
            "id": case.div1.attrs['id'],
            "text": " ".join(case.text.split()), # split on all whitespace, then join on " ", to remove long sequences of whitespace
            "any_defendant_female": False,
            "any_defendant_male": False,
            "any_victim_female": False,
            "any_victim_male": False,
        }
        if "offenceDescription" in descriptions:
            row["offenceText"] = descriptions["offenceDescription"].get("text", nan) # `dictionary.get(key, default)` is the same as `dictionary[key] if key in dictionary else default`
            row["offenceCategory"] = descriptions["offenceDescription"].get("offenceCategory", nan)
            row["offenceSubcategory"] = descriptions["offenceDescription"].get("offenceSubcategory", nan)
        if "verdictDescription" in descriptions:
            row["verdictText"] = descriptions["verdictDescription"].get("text", nan)
            row["verdictCategory"] = descriptions["verdictDescription"].get("verdictCategory", nan)
        if "punishmentDescription" in descriptions:
            row["punishmentText"] = descriptions["punishmentDescription"].get("text", nan)
            row["punishmentCategory"] = descriptions["punishmentDescription"].get("punishmentCategory", nan)
            row["punishmentSubcategory"] = descriptions["punishmentDescription"].get("punishmentSubcategory", nan)
        for person in people:
            if person.get("type") == "defendantName" and person.get("gender") == "female":
                row["any_defendant_female"] = True
            if person.get("type") == "defendantName" and person.get("gender") == "male":
                row["any_defendant_male"] = True
            if person.get("victim") == "victimName" and person.get("gender") == "female":
                row["any_victim_female"] = True
            if person.get("victim") == "victimName" and person.get("gender") == "male":
                row["any_victim_male"] = True
        rows.append(row)
    return pd.DataFrame(rows)

table = table_of_cases(trial_session_xmls)
table

,date,id,text,any_defendant_female,any_defendant_male,any_victim_female,any_victim_male,offenceText,offenceCategory,offenceSubcategory,verdictText,verdictCategory,punishmentText,punishmentCategory,punishmentSubcategory
0,17500117,f17500117-1,THE PROCEEDINGS ON THE King's Commissions of t...,False,False,False,False,wilful and corrupt perjury,deception,perjury,"The Prosecutor not appearing, he was acquitted",notGuilty,[Transportation. See summary.],transport,NaN
1,17500228,f17500228-1,THE PROCEEDINGS ON THE King's Commissions of t...,False,False,False,False,"stealing one silver watch, val. 50 s. the good...",theft,theftFromPlace,Guilty 39 s,guilty,[Transportation. See summary.],transport,NaN
2,17500425,f17500425-1,THE PROCEEDINGS ON THE King's Commissions of t...,False,False,False,False,wilful and corrupt perjury,deception,perjury,Guilty,guilty,imprisoned six months in Newgate,imprison,newgate
3,17500530,f17500530-1,THE PROCEEDINGS ON THE King's Commissions of t...,False,False,False,False,"stealing one silk handkerchief , value 12 d.",theft,grandLarceny,Guilty,guilty,[Transportation. See summary.],transport,NaN
4,17500711,f17500711-1,THE PROCEEDINGS ON THE King's Commissions of t...,False,False,False,False,"stealing three linen shirts, two table cloths",theft,other,Guilty,guilty,[Imprisonment. See summary.],imprison,houseOfCorrection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,17900708,f17900708-1,"The Monster. THE TRIAL OF Rhynwick, alias Renw...",False,False,False,False,"that he, on the 18th of January\n last, with f...",breakingPeace,wounding,GUILTY,guilty,sentence be respited,noPunish,sentenceRespited
340,17900710,f17900710-1,Horrid Conspiracy! THE TRIAL OF ROBERT JACQUES...,False,False,False,False,intending to injure and defraud \n ...,deception,fraud,GUILTY,guilty,pillory at the Royal Exchange for the space of...,corporal,pillory
341,17900915,f17900915-1,THE WHOLE PROCEEDINGS ON THE KING'S Commission...,False,False,False,False,"for a conspiracy, in causing \n ...",miscellaneous,pervertingJustice,NOT GUILTY,notGuilty,fined 1 s.,miscPunish,fine
342,17901027,f17901027-1,THE WHOLE PROCEEDINGS ON THE KING'S Commission...,False,False,False,False,uttering a counterfeit shilling,royalOffences,coiningOffences,NOT GUILTY,notGuilty,give security,miscPunish,sureties


***(4) Data preparation and EDA:***

In [ ]:
!pip install nltk svgling

**Tokenization**

In [ ]:
# using the split function to create tokens
text_1 = table['text'][0]
sentences = text_1.split(".")
for s in sentences[:5]:
    print(s + '\n')

THE PROCEEDINGS ON THE King's Commissions of the Peace, Oyer and Terminer, and Goal Delivery FOR THE CITY of LONDON; And also the Goal Delivery for the County of MIDDLESEX, HELD AT JUSTICE-HALL in the OLD-BAILEY

 On WEDNESDAY the 17th, THURSDAY the 18th, FRIDAY the 19th, and SATURDAY the 20th, of January

 In the 23d Year of His MAJESTY's Reign

 BEING THE Second SESSIONS in the MAYORALTY of the Rt

 Honble Sir Samuel Pennant , Knt



In [ ]:
tokens = text_1.split(" ")
tokens[:10]

['THE',
 'PROCEEDINGS',
 'ON',
 'THE',
 "King's",
 'Commissions',
 'of',
 'the',
 'Peace,',
 'Oyer']

**NLTK:**

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
text_2 = table['text'][1]
sents = nltk.sent_tokenize(text_2)
sents[:3]

["THE PROCEEDINGS ON THE King's Commissions of the Peace, Oyer and Terminer, and Goal Delivery FOR THE CITY of LONDON; And also the Goal Delivery for the County of MIDDLESEX, HELD AT JUSTICE-HALL in the OLD-BAILEY.",
 'On WEDNESDAY the 28th, of February.',
 'THURSDAY the 1st, FRIDAY the 2d, SATURDAY the 3d, MONDAY the 5th, TUESDAY the 6th, and WEDNESDAY the 7th of March.']

In [ ]:
nltk.word_tokenize(text_2)[:10]

['THE',
 'PROCEEDINGS',
 'ON',
 'THE',
 'King',
 "'s",
 'Commissions',
 'of',
 'the',
 'Peace']

In [ ]:
tokens = nltk.word_tokenize(text_2)
fd = nltk.collocations.FreqDist(tokens)
fd.most_common()[:10]

[(',', 1813),
 ('.', 924),
 ('the', 919),
 ('and', 535),
 ('of', 456),
 ('I', 393),
 ('was', 376),
 ('to', 322),
 ('in', 302),
 (';', 292)]

In [ ]:
def rem_punc_stop(text):
    
    from string import punctuation
    from nltk.corpus import stopwords
    
    stop_words = set(stopwords.words("english"))
    punctuation = set(punctuation)
    
    punc_free = "".join([ch for ch in text if ch not in punctuation])
    
    words = nltk.word_tokenize(punc_free)
    
    noise_free = [word for word in words if word not in stop_words]
    
    return noise_free

In [ ]:
from nltk.corpus import stopwords
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
nltk.download('stopwords')
tokens_reduced = rem_punc_stop(text_2)
tokens_reduced[:10]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['THE',
 'PROCEEDINGS',
 'ON',
 'THE',
 'Kings',
 'Commissions',
 'Peace',
 'Oyer',
 'Terminer',
 'Goal']

In [ ]:
type(tokens_reduced)
fd_reduced = nltk.collocations.FreqDist(tokens_reduced)
fd_reduced.most_common()[:10]

[('I', 385),
 ('one', 137),
 ('said', 103),
 ('John', 96),
 ('indicted', 89),
 ('goods', 71),
 ('prisoner', 71),
 ('two', 70),
 ('stealing', 67),
 ('William', 58)]

In [ ]:
nltk.download('averaged_perceptron_tagger')
tagged = nltk.pos_tag(tokens[2:8])
tagged

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[('ON', 'IN'),
 ('THE', 'NNP'),
 ('King', 'NNP'),
 ("'s", 'POS'),
 ('Commissions', 'NNS'),
 ('of', 'IN')]

**STEMMING:**

In [ ]:
snowball = nltk.SnowballStemmer('english')

In [ ]:
snowball.stem("guilty")

'guilti'

**TEXT PRE-PROCESSING:**

In [ ]:
def does_it_all(text):
    tokens = ""
    
    not_stemmed = rem_punc_stop(text)
    stemmed = [snowball.stem(word.lower()) for word in not_stemmed]
    for word in not_stemmed:
        tokens += snowball.stem(word) + " "
    
    return tokens

In [ ]:
table_tokenized = table.copy()
table_tokenized["tokens"] = table["text"].apply(does_it_all)
table_tokenized

,date,id,text,any_defendant_female,any_defendant_male,any_victim_female,any_victim_male,offenceText,offenceCategory,offenceSubcategory,verdictText,verdictCategory,punishmentText,punishmentCategory,punishmentSubcategory,tokens
0,17500117,f17500117-1,THE PROCEEDINGS ON THE King's Commissions of t...,False,False,False,False,wilful and corrupt perjury,deception,perjury,"The Prosecutor not appearing, he was acquitted",notGuilty,[Transportation. See summary.],transport,NaN,the proceed on the king commiss peac oyer term...
1,17500228,f17500228-1,THE PROCEEDINGS ON THE King's Commissions of t...,False,False,False,False,"stealing one silver watch, val. 50 s. the good...",theft,theftFromPlace,Guilty 39 s,guilty,[Transportation. See summary.],transport,NaN,the proceed on the king commiss peac oyer term...
2,17500425,f17500425-1,THE PROCEEDINGS ON THE King's Commissions of t...,False,False,False,False,wilful and corrupt perjury,deception,perjury,Guilty,guilty,imprisoned six months in Newgate,imprison,newgate,the proceed on the king commiss peac oyer term...
3,17500530,f17500530-1,THE PROCEEDINGS ON THE King's Commissions of t...,False,False,False,False,"stealing one silk handkerchief , value 12 d.",theft,grandLarceny,Guilty,guilty,[Transportation. See summary.],transport,NaN,the proceed on the king commiss peac oyer term...
4,17500711,f17500711-1,THE PROCEEDINGS ON THE King's Commissions of t...,False,False,False,False,"stealing three linen shirts, two table cloths",theft,other,Guilty,guilty,[Imprisonment. See summary.],imprison,houseOfCorrection,the proceed on the king commiss peac oyer term...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,17900708,f17900708-1,"The Monster. THE TRIAL OF Rhynwick, alias Renw...",False,False,False,False,"that he, on the 18th of January\n last, with f...",breakingPeace,wounding,GUILTY,guilty,sentence be respited,noPunish,sentenceRespited,the monster the trial of rhynwick alia renwick...
340,17900710,f17900710-1,Horrid Conspiracy! THE TRIAL OF ROBERT JACQUES...,False,False,False,False,intending to injure and defraud \n ...,deception,fraud,GUILTY,guilty,pillory at the Royal Exchange for the space of...,corporal,pillory,horrid conspiraci the trial of robert jacqu fo...
341,17900915,f17900915-1,THE WHOLE PROCEEDINGS ON THE KING'S Commission...,False,False,False,False,"for a conspiracy, in causing \n ...",miscellaneous,pervertingJustice,NOT GUILTY,notGuilty,fined 1 s.,miscPunish,fine,the whole proceed on the king commiss peac oye...
342,17901027,f17901027-1,THE WHOLE PROCEEDINGS ON THE KING'S Commission...,False,False,False,False,uttering a counterfeit shilling,royalOffences,coiningOffences,NOT GUILTY,notGuilty,give security,miscPunish,sureties,the whole proceed on the king commiss peac oye...


**Feature Extracting (TF-IDF)**

In [ ]:
text_list = table_tokenized['text']
text_list

0      THE PROCEEDINGS ON THE King's Commissions of t...
1      THE PROCEEDINGS ON THE King's Commissions of t...
2      THE PROCEEDINGS ON THE King's Commissions of t...
3      THE PROCEEDINGS ON THE King's Commissions of t...
4      THE PROCEEDINGS ON THE King's Commissions of t...
                             ...                        
339    The Monster. THE TRIAL OF Rhynwick, alias Renw...
340    Horrid Conspiracy! THE TRIAL OF ROBERT JACQUES...
341    THE WHOLE PROCEEDINGS ON THE KING'S Commission...
342    THE WHOLE PROCEEDINGS ON THE KING'S Commission...
343    THE PROCEEDINGS ON THE KING'S Commission of th...
Name: text, Length: 344, dtype: object

In [ ]:
tf = TfidfVectorizer(analyzer='word', 
                     ngram_range=(1,3), 
                     min_df = 0, 
                     stop_words = 'english')

In [ ]:
tfidf_matrix = tf.fit_transform(text_list)
feature_names = tf.get_feature_names()
len(feature_names)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


6411778

In [ ]:
dense = tfidf_matrix.todense()
document = dense[0].tolist()[0]
phrase_scores = [pair for pair in zip(range(0, len(document)), document) if pair[1] > 0]

In [ ]:
sorted_phrase_scores = sorted(phrase_scores, key=lambda t: t[1] * -1)
for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sorted_phrase_scores][:20]:
    print('{0: <20} {1}'.format(phrase, score))

***(5) Classification:***

In [ ]:
# Training, Validation, Test Sets

# X
X = table_tokenized['tokens']
tf = TfidfVectorizer(analyzer='word', 
                     ngram_range=(1,3), 
                     min_df = 0, 
                     stop_words = 'english')

tfidf_matrix =  tf.fit_transform(X)

#y

y = table_tokenized['punishmentCategory']

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, y,
                                                   train_size = .80,
                                                   test_size = .20)

# Train/Validation Split
X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train,
                                                           train_size = .75,
                                                           test_size = .25)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
def metrics(true, pred):
  print("The accuracy for the model is: ", accuracy_score(true, pred))
  print("The f1 score for the model is: ", f1_score(true, pred, average='weighted'))
  

**Naive Bayes**

In [ ]:
nb = MultinomialNB()
nb_model = nb.fit(X_train, y_train)
nb_pred = nb_model.predict(X_validate)
np.mean(nb_pred == y_validate)
nb_cf_matrix = confusion_matrix(y_validate, nb_pred)

In [ ]:
metrics(y_validate, nb_pred)

In [ ]:
nb_df_cm = pd.DataFrame(nb_cf_matrix, range(4), range(4))
plt.figure(figsize = (10,7))
sn.set(font_scale=1.4)#for label size
sn.heatmap(nb_df_cm, 
           annot=True,
           annot_kws={"size": 16})

plt.title("Naive Bayes Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

**Multinomial Logistic Regression**

In [ ]:
logreg = linear_model.LogisticRegression(solver = "newton-cg", multi_class = 'multinomial', class_weight = 'balanced')
log_model = logreg.fit(X_train, y_train)
log_pred = log_model.predict(X_validate)
np.mean(log_pred == y_validate)
log_cf_matrix = confusion_matrix(y_validate, log_pred)

In [ ]:
metrics(y_validate, log_pred)

In [ ]:
log_df_cm = pd.DataFrame(log_cf_matrix, range(4),range(4))
plt.figure(figsize = (10,7))
sn.set(font_scale=1.4)#for label size
sn.heatmap(log_df_cm, 
           annot=True,
           annot_kws={"size": 16})

plt.title("Multinomial Logistic Regression Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Labels")
plt.show()

**SVM**

In [ ]:
from sklearn import svm

In [ ]:
svm = svm.LinearSVC(class_weight = 'balanced')
svm_model = svm.fit(X_train, y_train)

In [ ]:
svm_pred = svm_model.predict(X_validate)
np.mean(svm_pred == y_validate)
svm_cf_matrix = confusion_matrix(y_validate, svm_pred)

In [ ]:
metrics(y_validate, svm_pred)

In [ ]:
svm_df_cm = pd.DataFrame(svm_cf_matrix, range(4),
                  range(4))
plt.figure(figsize = (10,7))
sn.set(font_scale=1.4)#for label size
sn.heatmap(svm_df_cm, 
           annot=True,
           annot_kws={"size": 16})

plt.title("SVM Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

**Test Final Classifier**


In [ ]:
log_test_pred = log_model.predict(X_test)

np.mean(log_test_pred == y_test)

In [ ]:
metrics(y_validate, log_test_pred)

In [ ]:
log_cf_matrix = confusion_matrix(y_test, log_test_pred)

svm_df_cm = pd.DataFrame(svm_cf_matrix, range(4),
                  range(4))
plt.figure(figsize = (10,7))
sn.set(font_scale=1.4)#for label size
sn.heatmap(svm_df_cm, 
           annot=True,
           annot_kws={"size": 16})

plt.title("SVM Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

In [ ]:
coef = svm.coef_[0]
coef

**Feature Extraction**

In [ ]:
table_tokenized['clean_text'] = table_tokenized['text'].str.lower()
table_tokenized['words'] = table_tokenized['clean_text'].apply(nltk.word_tokenize)
table_tokenized['sentences'] = table_tokenized['clean_text'].apply(nltk.sent_tokenize)
table_tokenized

In [ ]:
table_tokenized['char_count'] = table_tokenized['clean_text'].str.len()
table_tokenized['word_count'] = table_tokenized['words'].str.len()
table_tokenized['sentence_count'] = table_tokenized['sentences'].str.len()
table_tokenized['avg_word_length'] = table_tokenized['char_count'] / table_tokenized['word_count']
table_tokenized['avg_sentence_length'] = table_tokenized['word_count'] / table_tokenized['sentence_count']
table_tokenized

In [ ]:
topics = ['prisoner', 'wife', ' stealing',
          'indicted']

for i in topics:    
    table_tokenized[i] = table_tokenized['clean_text'].str.count(i)


In [ ]:
table_tokenized.describe()